# Clustering and classifying industries

In [3]:
# import packages
import datetime
import scipy.integrate as integrate
import math
import numpy as np
import operator as op
import os
import pandas as pd
from functools import reduce
from scipy.optimize import minimize

In [53]:
os.getcwd()

'/Users/sahra/Google Drive/LSE/ST442 - BML/Project'

## Data Preprocessing

We start by reading in the data, cleaning it and extracting the relevant features.

In [9]:
# import data
chem = pd.read_excel(io="chemicals.xlsx")
soft = pd.read_excel(io="software.xlsx")

In [10]:
chem[1:100].describe()

,PETRONAS CHEMICALS GP.,MITSUI CHEMICALS,TATA CHEMICALS,ABA CHEMICALS 'A',BODAL CHEMICALS,CAPROLACTAM CHEMICALS,CHD CHEMICALS,CHEMBOND CHEMICALS,DYMATIC CHEMICALS 'A',GARODIA CHEMICALS,...,JAPAN PURE CHEMICAL,JIANGSU FLAG CHEMICAL 'A',JIANGSU YIDA CHEMICAL 'A',JINAN ACETATE CHEMICAL,JIUTIAN CHEMICAL GP.,JORDAN CHEMICAL INDS. SUSP - SUSP.01/04/19,KAWAGUCHI CHEMICAL IND.,KG CHEMICAL,KO YO CHEMICAL (GROUP),KODAMA CHEMICAL IND.
count,81.000000,99.000000,99.000000,71.000000,99.000000,99.000000,16.000000,99.000000,99.000000,68.000000,...,99.000000,8.000000,0.0,21.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000
mean,6.520123,1621.515152,368.880808,7.588451,40.817778,10.256768,9.601250,119.068990,8.673131,39.886912,...,2348.616162,57.516250,NaN,126.038095,0.065192,1.935253,1203.232323,13122.929293,0.692495,695.858586
std,0.504367,564.724267,94.343055,3.310733,43.587939,9.817166,1.911645,62.443197,2.890721,7.693008,...,363.749233,3.798063,NaN,27.306144,0.048240,0.515697,279.211091,3953.066558,0.432968,163.513777
min,5.130000,765.000000,216.350000,2.650000,6.000000,3.040000,7.780000,32.000000,4.420000,27.000000,...,1807.000000,51.380000,NaN,88.300000,0.012000,1.320000,740.000000,6200.000000,0.111000,430.000000
25%,6.240000,1237.500000,312.750000,4.630000,10.375000,3.040000,8.260000,75.750000,5.875000,31.950000,...,2107.500000,55.820000,NaN,107.500000,0.033500,1.520000,1010.000000,9655.000000,0.410000,555.000000
50%,6.610000,1405.000000,342.350000,7.730000,29.800000,6.300000,8.720000,91.150000,8.580000,43.350000,...,2305.000000,56.540000,NaN,116.000000,0.054000,1.750000,1110.000000,13750.000000,0.605000,660.000000
75%,6.800000,1885.000000,412.625000,10.020000,41.350000,13.875000,10.330000,181.675000,10.950000,46.100000,...,2497.500000,60.525000,NaN,144.000000,0.082000,2.240000,1385.000000,16275.000000,0.845000,820.000000
max,7.620000,3110.000000,656.800000,17.650000,179.850000,50.000000,14.000000,251.000000,15.800000,48.490000,...,3800.000000,62.550000,NaN,181.000000,0.260000,3.040000,2220.000000,23450.000000,2.660000,1080.000000


In [11]:
soft[1:100].describe()

,CONSTELLATION SOFTWARE,TABLEAU SOFTWARE CL.A,GUIDEWIRE SOFTWARE,PAYCOM SOFTWARE,ULTIMATE SOFTWARE GP.,COUPA SOFTWARE,CYBER ARK SOFTWARE,DHC SOFTWARE 'A',SOFTWARE N,ALFA FINANCIAL SOFTWARE HOLDINGS(WI),...,FLEXI INTL.SFTW.,FUJIAN RONGJI SFTW.'A',INDO-PAC.SFTW.& ENTM.,INNVN.SFTW.EXPS.,JIANGSU HOPERUN SFTW.'A',RENOWORKS SFTW.,BEIJING JOIN-CHEER SFTW. 'A',CYBERTECH SYS.& SFTW.,INDIAN INFOTECH & SFTW.,SICHUAN JIUYUAN YINHAI SFTW.'A'
count,99.000000,51.000000,67.000000,40.000000,99.000000,10.000000,35.000000,99.000000,99.000000,3.000000,...,99.000000,83.000000,99.000000,99.000000,61.000000,99.000000,96.000000,99.000000,85.000000,19.000000
mean,256.099495,69.885294,47.294925,37.820000,116.200657,28.295000,48.020857,7.384343,28.597919,455.500000,...,0.200928,9.745060,1.891111,2.748586,11.773443,0.147778,7.904792,29.754040,7.598941,44.246842
std,222.658288,19.549266,12.225728,15.129718,64.061728,3.330757,8.378891,3.991865,5.920462,27.086666,...,0.129971,4.802298,1.307209,2.131873,5.949346,0.150710,5.528814,21.596027,11.293855,8.932433
min,32.600000,41.680000,17.830000,12.880000,20.310000,24.570000,27.390000,2.330000,16.373000,429.750000,...,0.040000,4.020000,0.480000,0.550000,3.470000,0.010000,2.130000,9.900000,0.160000,33.110000
25%,69.450000,53.580000,39.220000,29.800000,54.065000,25.672500,42.890000,4.000000,24.931500,441.375000,...,0.110000,6.140000,0.940000,0.950000,6.590000,0.030000,3.760000,13.425000,0.270000,35.835000
50%,145.410000,65.490000,49.330000,36.615000,117.290000,27.100000,48.350000,5.450000,27.971000,453.000000,...,0.160000,7.510000,1.560000,1.890000,11.600000,0.050000,5.270000,17.950000,1.660000,43.440000
75%,503.180000,81.110000,56.215000,47.710000,170.790000,30.557500,52.305000,10.505000,32.499000,468.375000,...,0.222500,12.700000,2.170000,3.965000,15.500000,0.290000,12.670000,43.850000,13.630000,50.650000
max,693.400000,113.810000,71.920000,70.460000,227.210000,34.620000,66.500000,21.470000,42.507000,483.750000,...,0.550000,26.020000,5.850000,9.610000,31.830000,0.570000,28.300000,92.700000,44.750000,65.750000


In [12]:
# drop columns with at least 1% missing values
chem = chem[chem.columns[chem.isnull().mean() < 0.01]]
soft = soft[soft.columns[soft.isnull().mean() < 0.01]]

# drop columns with zero values
chem = chem[chem>0]
soft = soft[soft>0]

In [160]:
def indices(data, frequency, daily_data = False):
    """ Gives indices of days that singalize the beginning of new period
    Args: 
        daily_data - data set with datetime64-type column Name
        frequency - monthly, quarterly or annual
    Returns: 
        indices - list of vector indices that are the first day in a 
        month, quarter or year 
    """
    dates = data["Name"]
    months = np.zeros(len(dates))
    years = np.zeros(len(dates))
    for i in range(len(dates)):
        months[i] = dates[i].month
        years[i] = dates[i].year
    old_month = 0
    old_year = 0
    indices = [0]
    
    for i in range(len(years)):
        if frequency == "annually" or frequency == "biannually":
            current_year = years[i]
            if current_year != years[old_year]:
                indices.append(i)
                old_year = i
                
    else:
        for i in range(len(months)):
            current_month = months[i]
            if current_month != months[old_month]:
                old_month = i
                if frequency=="monthly":
                    indices.append(i)
                elif frequency=="quarterly":
                    if (current_month==1 or current_month==4 or 
                        current_month==7 or current_month==10):
                        indices.append(i)
    
    if frequency == "biannually":
            indices = indices[::2]
        
    return(indices)


def returns(daily_data, frequency, drop = True, vol = False):
    """ Extracts attributes from daily data  
        Args: 
            daily_data - daily data as pandas dataframe
            frequency - daily, monthly, quarterly, annual
            drop - if 'True', drop companies with more than 10% NaN values
            vol - if 'True', computes standard deviation instead of mean 
         Returns: 
            return - monthly, quarterly or annual return resp. average data in pandas Dataframe
    """
    daily_data_df = daily_data
    if frequency != "daily":
        indices_data = indices(daily_data, frequency)
        daily_data = daily_data.values
        returns = np.zeros((len(indices_data)-1,daily_data.shape[1]-1))
        for i in range(len(indices_data)-1):
            if vol==False:
                returns[i,:] = np.divide(daily_data[indices_data[i+1],1:], 
                                          daily_data[indices_data[i],1:]) - 1
            else:
                returns[i,:] = [np.std(daily_data[range(indices_data[i],indices_data[i+1]),j]) for j in range(1,daily_data.shape[1])]
        if drop==True:
            # delete columns if more than 10% of the returns are zero
            returns_zero = [np.mean(returns[:,i]==0) 
                            for i in range(returns.shape[1])]

            returns = returns[:,[returns_zero[i]<0.1 
                                 for i in range(len(returns_zero))]]

            dropped = [returns_zero[i]<0.1 for i in range(len(returns_zero))]
            dropped = [i*(dropped[i]==True) for i in range(len(returns_zero))]
            dropped = np.transpose(dropped)
            dropped = dropped[dropped!=0]
            return(pd.DataFrame(returns, columns=daily_data_df.columns.values[dropped]))

        else:
            return(pd.DataFrame(returns, columns=daily_data_df.columns.values[1:]))
    else:
        daily_data = daily_data.values
        return([np.std(daily_data[:,j]) for j in range(1,daily_data.shape[1])])

As features we regard the quarterly and annual returns and annual volatility of the period regarded. We save these measures in the arrays 'merged_chem' and 'merged_tech'.

In [220]:
# merge quarterly and annual returns (in  numpy array) for chemical and technical industry
merged_chem = np.row_stack((returns(chem,"annually",drop=False),
                            returns(chem,"annually",drop=False,vol=True)))
merged_soft = np.row_stack((returns(soft,"annually",drop=False),
                            returns(soft,"annually",drop=False,vol=True)))

In [221]:
# delete columns if any attribute is NA
zero_clms = [np.mean(merged_chem[:,i]==0) 
                for i in range(merged_chem.shape[1])] 
merged_chem = merged_chem[:,[zero_clms[i]<0.1 
                     for i in range(len(zero_clms))]]

zero_clms = [np.mean(merged_soft[:,i]==0) 
                for i in range(merged_soft.shape[1])] 
merged_soft = merged_soft[:,[zero_clms[i]<0.1 
                     for i in range(len(zero_clms))]]

We will only analyse the 100 best performing stocks.

In [222]:
# merge merged_chem and merged_tech
merged_data = np.column_stack((merged_chem, merged_soft))
merged_classes = np.concatenate((np.zeros(merged_chem.shape[1]), 
                                 np.zeros(merged_soft.shape[1])+1))

# delete columns with too many zero returns
merged_zero = [np.mean(merged_data[:,i]==0) 
               for i in range(merged_data.shape[1])]

merged_classes = merged_classes[[merged_zero[i]<0.1 
                                 for i in range(merged_data.shape[1])]]

merged_data = merged_data[:,[merged_zero[i]<0.1 
                             for i in range(merged_data.shape[1])]] 

In [223]:
merged_data.shape

(20, 152)

In [285]:
# merge merged_chem and merged_tech
merged_data = np.column_stack((merged_chem, merged_soft))
merged_classes = np.concatenate((np.zeros(merged_chem.shape[1]), 
                                 np.zeros(merged_soft.shape[1])+1))
# delete columns with too many zero returns
merged_zero = [np.mean(merged_data[:,i]==0) 
               for i in range(merged_data.shape[1])]

merged_classes = merged_classes[[merged_zero[i]<0.1 
                                 for i in range(merged_data.shape[1])]]

merged_data = merged_data[:,[merged_zero[i]<0.1 
                             for i in range(merged_data.shape[1])]] 

# 120 stocks with best performance in period under consideration
take_in = returns(chem,"annually",drop=False).shape[0]
total_merged = np.array([np.prod(1+merged_data[range(take_in),i]) 
                         for i in range(merged_data.shape[1])])  

top120 = total_merged.argsort()[-120:][::-1]
top120_data = np.transpose(merged_data[:,top120])
top120_classes = merged_classes[top120]

Before we analyse the data further, we normalize the features by substracting the mean and dividing by the standard deviation.

In [286]:
def norm_it(data):
    """Standardizes input"""
    mean = [np.mean(data[:,i]) for i in range(data.shape[1])]
    std = [np.std(data[:,i]) for i in range(data.shape[1])]
    normed = np.array([[(data[i,j] - mean[j])/std[j] for j in range(data.shape[1])] for i in range(data.shape[0])])
    return(normed)

top120_data = norm_it(top120_data)

In [315]:
import random
random.seed(451)

# extract train and test data according to 70:50 split
train_indices = random.sample(range(120),100)
test_indices = list(set(range(120)) - set(train_indices))
y_train, X_train = top120_classes[train_indices], top120_data[train_indices,:]
y_test, X_test = top120_classes[test_indices], top120_data[test_indices,:]

### Multi collinearity  analysis

In [316]:
# compute eigenvalues in order to avoid multi-collinearity
p = X_train.shape[1]
p_new = p
corr = np.corrcoef(X_train, rowvar=0)
w, v = np.linalg.eig(corr)       
[np.round(w[i],4) for i in range(p_new)]

[9.9014,
 2.0474,
 1.4839,
 1.4332,
 1.1012,
 0.875,
 0.839,
 0.7077,
 0.6003,
 0.4299,
 0.3436,
 0.1457,
 0.062,
 0.0121,
 0.0081,
 0.0048,
 0.0035,
 0.0011,
 0.0002,
 0.0001]

In [317]:
X_train =  X_train[:,:p_new]
X_test = X_test[:,:p_new]

## Descriptive Statistics

In [318]:
print("We consider {} stocks with {} features during the period from {} to {}.".format(top120_data.shape[0], X_train.shape[1], chem['Name'][0].date(), chem['Name'][chem.shape[0]-1].date()))

We consider 120 stocks with 20 features during the period from 2009-04-29 to 2019-04-29.


In [319]:
print("Among the 120 best performing stocks of the past two years we have {} stocks from the software sector.".format(np.sum(top120_classes)))

Among the 120 best performing stocks of the past two years we have 55.0 stocks from the software sector.


We assume that the classification of the selected stocks $x_1,...,x_n$  with $n=100$ for the event $Y=1$, i.e. the stock belongs to technology industry, follows a binomial distribution with parameter $p$. 

In [320]:
n = len(top120_classes)
p = X_train.shape[1]
y = np.sum(top120_classes)

# frequentist descriptive statistics

p_mean = y/n
p_mode = y/n
p_median = y/n
p_std = p_mean*(1-p_mean)/n
p_conf095 = [p_mean/n - 1.96*p_std,p_mean/n + 1.96*p_std]

In [321]:
# Bayesian descriptive statistics

def nck(n, k):
    """n chooses k"""
    n = int(n)
    k = int(k)
    r = min(k, n-k)
    num = reduce(op.mul, range(n, n-r, -1), 1)
    den = reduce(op.mul, range(1, r+1), 1)
    return num / den

def posterior(theta):
    """Computes the posterior pi(theta|y) given y is binomially distributed
    and theta uniformly distributed on [0.5,1] up to proportionality"""
    if (theta > 0.5) and (theta < 1):
        return(theta**y*(1-theta)**(n-y))
    else: 
        return(0)

def help_exp(theta):
    """pi(theta|y)*theta given y is binomially distributed and theta 
    uniformly distributed on [0.5,1] up to proportionality"""
    if (theta > 0.5) and (theta < 1):
        return(theta**(y+1)*(1-theta)**(n-y))
    else: 
        return(0)

# posterior mean with integral
integral_posterior = integrate.quad(posterior,0.5,1)[0]
save = integrate.quad(help_exp,0.5,1)[0]
post_mean = save/integral_posterior

# posterior mean and median based on Monte Carlo
size = 1000000
sample = np.random.beta(a=y+1,b=n-y+1,size=size)
sample = sample[[sample[i]>0.5 for i in range(size)]]
post_mean = np.mean(sample)
post_median = np.median(sample)
credint095 = [np.quantile(sample,0.025),np.quantile(sample,0.975)]
# posterior mode based on optimization
thetas = [i/2000+0.5 for i in range(1000)]
posteriors = [posterior(thetas[i]) for i in range(1000)]
post_mode = thetas[np.argmax(posteriors)]

In [322]:
estimates_df = pd.DataFrame(data={'Frequentist estimators':
  [p_mean,p_mode,p_median,p_conf095[0],p_conf095[1]],'Bayesian estimators':
  [post_mean,post_mode,post_median,credint095[0],credint095[1]]},
  index=['mean', 'mode', 'median', 'lower 5% bound', 'upper 5% bound'])

In [323]:
print(estimates_df.to_latex())

\begin{tabular}{lrr}
\toprule
{} &  Frequentist estimators &  Bayesian estimators \\
\midrule
mean           &                0.458333 &             0.524557 \\
mode           &                0.458333 &             0.500500 \\
median         &                0.458333 &             0.519343 \\
lower 5\% bound &               -0.000236 &             0.500777 \\
upper 5\% bound &                0.007874 &             0.576749 \\
\bottomrule
\end{tabular}



## Classification

### Linear Discriminant analysis

In [324]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis(store_covariance=True)
lda.fit(X_train, y_train)
lda.means_, lda.priors_, lda.covariance_, np.sqrt(np.diag(lda.covariance_))

(array([[-0.06338019, -0.02680527,  0.00580608, -0.11511457, -0.13036258,
         -0.04689206, -0.14040496,  0.18364797, -0.08808063, -0.18765357,
          0.21588743,  0.17272398,  0.19378715,  0.17191618,  0.18826972,
          0.20893629,  0.22136161,  0.19662697,  0.2206557 ,  0.16382194],
        [-0.0014257 , -0.01820816, -0.03233294,  0.17122438,  0.21197888,
          0.04299077,  0.18434816, -0.23885735,  0.14331788,  0.10018602,
         -0.18850588, -0.14338712, -0.16958257, -0.15237958, -0.15940688,
         -0.17898903, -0.18935873, -0.1853969 , -0.1935005 , -0.13752094]]),
 array([0.54, 0.46]),
 array([[ 1.00961554, -0.21826781, -0.03990405, -0.02719648, -0.0536796 ,
          0.01865632, -0.08148403,  0.13981409,  0.32071437, -0.42655374,
         -0.07849852, -0.0625894 , -0.10638233, -0.06399604, -0.06722267,
         -0.06021107, -0.07283895, -0.0572483 , -0.07002818, -0.02651736],
        [-0.21826781,  1.03474369, -0.10393618,  0.26884933, -0.15707337,
          0

In [325]:
lda.fit(X_train, y_train).score(X_train, y_train), lda.fit(X_train, y_train).score(X_test, y_test)

(0.74, 0.7)

In [326]:
# forward subset selection
best_k_features = np.zeros((p,p), bool)
vals_of_k = np.zeros((p), bool)
for k in range(p):
    best = 0.5
    best_new =  -1
    for l in [i for i, x in enumerate(~best_k_features[k,:]) if x]:
        best_k_features[range(k,p),l] = True
        new_score =  lda.fit(X_train[:,best_k_features[k,:]], y_train).score(X_train[:,best_k_features[k,:]], y_train)
        if new_score > best:
            best = new_score
            if best_new != -1:
                best_k_features[range(k,p),best_new] = False  
            best_new = l
        else:
            best_k_features[range(k,p),l] = False
print(best_k_features)

[[False False False False False False False False  True False False False
  False False False False False False False False]
 [False False False False False False False False  True  True False False
  False False False False False False False False]
 [False False False False False False False False  True  True  True False
  False False False False False False False False]
 [False False False False False False False False  True  True  True  True
  False False False False False False False False]
 [False False False False  True False False False  True  True  True  True
  False False False False False False False False]
 [False False False False  True False False False  True  True  True  True
   True False False False False False False False]
 [False False False False  True False False False  True  True  True  True
   True False  True False False False False False]
 [False False False False  True False False False  True  True  True  True
   True False  True  True False False False False]


In [327]:
# 5-fold CV for best models with k features
k_cv =  5
n_train = y_train.shape[0]
best_k = 0
best_cv_score = 0.5
length_cv = int(n_train/k_cv)
for k in range(1,p):
    cv_score = 0
    selected_features = best_k_features[k,:]
    for n_cv in range(k_cv):
        indices_cv = np.zeros(n_train)!=0
        indices_cv[np.full((length_cv),length_cv*n_cv)+range(length_cv)]=True 
        y_train_cv,y_test_cv = y_train[indices_cv],y_train[~indices_cv]
        X_train_cv,X_test_cv = X_train[indices_cv,:][:,selected_features], X_train[~indices_cv,:][:,selected_features]
        cv_score += lda.fit(X_train_cv, y_train_cv).score(X_test_cv, y_test_cv)
    cv_score /= k_cv
    if cv_score > best_cv_score:
        best_cv_score = cv_score
        best_k = k

/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/anaconda3/lib/python3.7/site-packages/sklearn

In [328]:
best_k, best_cv_score # best cv feature number and validation error

(7, 0.6)

In [329]:
take_in = best_k_features[best_k,:]
# take_in = (True, True, False, False, False, False, False, False, True, False, False, False, False)

In [330]:
take_in

array([False, False, False, False,  True, False, False, False,  True,
        True,  True,  True,  True, False,  True,  True, False, False,
       False, False])

In [331]:
# training error for best cv feature number 
lda.fit(X_train[:,take_in], y_train).score(X_train[:,take_in], y_train)

0.7

In [332]:
# test error for best cv feature number 
lda.fit(X_train[:,best_k_features[best_k,:]], y_train).score(X_test[:,best_k_features[best_k,:]], y_test)

0.5

### Logistic Regression

In [333]:
import statsmodels.api as sm

# forward subset selection
best_k_features = np.zeros((p,p), bool)
vals_of_k = np.zeros((p), bool)
for k in range(p):
    print(k)
    best = 0.5
    best_new =  -1
    for l in [i for i, x in enumerate(~best_k_features[k,:]) if x]:
        best_k_features[range(k,p),l] = True
        Xt = sm.add_constant(X_train[:,best_k_features[k,:]])
        new_score =  sm.Logit.fit(Xt, y_train).score(Xt, y_train)
        if new_score > best:
            best = new_score
            if best_new != -1:
                best_k_features[range(k,p),best_new] = False  
            best_new = l
        else:
            best_k_features[range(k,p),l] = False
print(best_k_features)

#model = sm.Logit(yt, Xt).fit()
#model.summary()

0


TypeError: super(type, obj): obj must be an instance or subtype of type

In [ ]:
# 6-fold CV for best models with k features
k_cv =  6
n_train = y_train.shape[0]
best_k = 0
best_cv_score = 0.3
length_cv = int(n_train/k_cv)
for k in range(1,p):
    cv_score = 0
    selected_features = best_k_features[k,:]
    for n_cv in range(k_cv):
        indices_cv = np.zeros(n_train)!=0
        indices_cv[np.full((length_cv),length_cv*n_cv)+range(length_cv)]=True 
        y_train_cv,y_test_cv = y_train[indices_cv],y_train[~indices_cv]
        X_train_cv,X_test_cv = X_train[indices_cv,:], X_train[~indices_cv,:]
        X_train_cv,X_test_cv = sm.add_constant(X_train_cv[:,selected_features]), sm.add_constant(X_test_cv[:,selected_features])     
        cv_score += sm.Logit.fit(X_train_cv, y_train_cv).score(X_test_cv, y_test_cv)
    cv_score /= k_cv
    if cv_score > best_cv_score:
        best_cv_score = cv_score
        best_k = k

In [ ]:
def neg_post(beta, x, y, m0, S0):
    logprior =  - .5* (beta-m0).T @ S0 @ (beta-m0)
    return  nll(beta, x, y) -  logprior

n_train,p =X_train.shape
m0 = np.zeros(p)
S0 = X_train.T.dot(X_train)/n_train
neg_post(beta_mle, X_train, y_train, m0, S0)

import scipy.optimize as opt

beta0 = np.ones(p) * .1
myoptions={'disp':True}
results = opt.minimize(neg_post,beta0,args=(X_train, y_train, m0, S0),
options = myoptions)
beta_map = results.x
print(beta_map)

def laplace_prec(beta, x, S0):
    n = x.shape[0]  
    S = np.diag(sigma(x,beta)*(1-sigma(x,beta)))
    prec = S0.values + x.T.dot(S.dot(x))    
    return prec

S0 = X_train.T.dot(X_train)/n_train #linear regression unit information prior
#get posterior covariance evaluating at beta_map 
prec = laplace_prec(beta_map, X_train, S0)
cov = sc.linalg.inv(prec)

#95% credible intervals
se = np.sqrt(np.diag(cov))
lower95 = beta_map - 1.96*se
upper95 = beta_map + 1.96*se

#present the output via a pandas data frame
results = np.column_stack([beta_map,se,lower95,upper95])
col = ['post mean','post se','lower 95% bound','upper 95% bound']
ind = ['intercept','age','nodes_detected']
results = pd.DataFrame(results,columns = col,index=ind)
results